In [0]:
# 1. Obter os parâmetros de entrada do job
catalog_name = dbutils.widgets.get("catalog")
schema_name = dbutils.widgets.get("schema")
folder_volume = dbutils.widgets.get("folder_volume") # Ex: api_meltano_20250617_2230

# 2. Configurar os caminhos e o Spark
volume_name = "raw"
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")

# Caminho base onde todos os arquivos JSON da execução foram salvos
source_path_base = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/{folder_volume}"
print(f"Iniciando processamento de JSON para a pasta: {source_path_base}")

# 3. Listar todos os arquivos .jsonl que foram enviados nesta execução
try:
    # Vamos procurar por arquivos que terminam com .jsonl
    json_files = [f.path for f in dbutils.fs.ls(source_path_base) if f.path.endswith('.jsonl')]
    if not json_files:
        print(f"Aviso: Nenhum arquivo .jsonl encontrado em '{source_path_base}'.")
        dbutils.notebook.exit("Nenhum arquivo para processar.")
except Exception as e:
    print(f"Erro ao listar arquivos em '{source_path_base}'. Verifique se a pasta e os arquivos foram enviados. Erro: {e}")
    dbutils.notebook.exit("Falha ao encontrar o diretório de origem.")

# 4. Loop para processar cada arquivo JSON
for file_path in json_files:
    
    # 5. Extrair o nome do arquivo para criar o nome da tabela
    # Ex: '.../purchaseorderdetail.jsonl' -> 'purchaseorderdetail'
    file_name = file_path.split('/')[-1]
    table_base_name = file_name.split('.')[0]
    
    # Constrói o nome final da tabela Delta: 'raw_api_purchaseorderdetail'
    table_name_final = f"raw_api_{table_base_name.lower()}"
    
    print(f"--- Processando Arquivo ---")
    print(f"Origem JSONL: {file_path}")
    print(f"Destino Delta: {table_name_final}")

    # 6. Ingestão dos dados usando COPY INTO (ideal para JSON)
    try:
        # COPY INTO é excelente para JSON, pois pode inferir o schema e lidar com evoluções.
        # 'force' = 'true' permite que ele sobrescreva uma tabela se necessário, útil para reprocessamento.
        # 'mergeSchema' = 'true' permite adicionar novas colunas se o schema do JSON mudar.
        spark.sql(f"""
            COPY INTO {catalog_name}.{schema_name}.{table_name_final}
            FROM (SELECT * FROM json.`{file_path}`)
            FILEFORMAT = JSON
            COPY_OPTIONS ('force' = 'true', 'mergeSchema' = 'true')
        """)
        print(f"Sucesso ao usar COPY INTO para a tabela {table_name_final}.")

    except Exception as e:
        # Fallback para criar a tabela com Spark Read/Write se COPY INTO falhar (ex: erro de sintaxe)
        print(f"COPY INTO falhou, tentando criar com Read/Write. Erro: {e}")
        try:
            # Opção 'primitivesAsString'='true' ajuda a evitar erros de schema misto lendo tudo como texto inicialmente.
            # 'multiLine'='true' é necessário se cada arquivo contiver um único objeto JSON formatado em várias linhas.
            # Se for JSONL (um objeto por linha), você pode remover a opção multiLine.
            df = spark.read.option("primitivesAsString", "true").option("multiLine", "true").json(file_path)
            
            # A função a seguir "achata" (flattens) o schema se ele for aninhado.
            # Isso transforma {"user": {"id": 1}} em colunas user_id = 1.
            def flatten_df(nested_df):
                from pyspark.sql.functions import col
                from pyspark.sql.types import StructType

                flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
                nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

                flat_df = nested_df.select(flat_cols +
                                           [col(nc + '.' + c).alias(nc + '_' + c)
                                            for nc in nested_cols
                                            for c in nested_df.select(nc + '.*').columns])
                return flat_df

            df_flat = flatten_df(df)

            df_flat.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{catalog_name}.{schema_name}.{table_name_final}")
            print(f"Sucesso ao criar a tabela {table_name_final} com Read/Write e achatamento de schema.")
        except Exception as e_write:
            print(f"ERRO: Falha crítica ao processar o arquivo {file_path} para a tabela {table_name_final}. Erro: {e_write}")
            continue # Pula para o próximo arquivo em caso de erro

print("--- Processamento de todos os arquivos JSON concluído. ---")